# XML / Mondial Exercise
## 1) Find the 10 countries with the lowest infant mortality rates

In [1]:
import pandas as pd
import numpy as np
from lxml import etree

In [2]:
document = etree.parse( './data/mondial_database.xml' )
document = document.getroot()

In [3]:
#iterate through each country to create dictionary for each country name (key) and corresponding infant mortality rate (value)
countries = []
mortality_rates = []

morts = document.findall('./country/infant_mortality')
for mort in morts:
    mortality_rates.append(float(mort.text))
    #using iterancestors to work backwards from 'infant_mortality' ensures there are no Nil values
    for country in mort.iterancestors('country'):
        countries.append(country.find('name').text)

In [4]:
#read dictionary into Pandas dataframe for manipulation
df = pd.DataFrame({'countries': countries, 'mortality_rates': mortality_rates})

In [5]:
#sort
df = df.sort_values('mortality_rates')

In [6]:
#get top 10
df.head(10)

,countries,mortality_rates
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


## 2) Find the 10 cities with the largest population

In [7]:
cities = []
populations = []

#use Xpath positional argument to ensure that population is the most recent census
pops = document.findall('./country/city/population[last()]')
for pop in pops:
    populations.append(int(pop.text))
    #use lxml iterancestors to work backwards and only include cities with population counts
    for city in pop.iterancestors('city'):
        cities.append((city.find('name').text))

In [8]:
#as before, read dictionary into Pandas dataframe
df = pd.DataFrame({'city': cities, 'population': populations})

In [9]:
#sort
df = df.sort_values('population', ascending = False)

In [10]:
#get the top 10
df.head(10)

,city,population
165,Seoul,9708483
154,Al Qahirah,8471859
75,Bangkok,7506700
123,Hong Kong,7055071
87,Ho Chi Minh,5968384
201,Singapore,5076700
153,Al Iskandariyah,4123869
205,New Taipei,3939305
166,Busan,3403135
102,Pyongyang,3255288


## 3) Find the 10 ethnic groups with largest overall populations

In [11]:
#create lists for name of each ethnic group, the percentage each group is of each countries' population, and the size of that population
populations = []
ethnic_groups = []
percentages = []

for country in document.findall('country'):
    pops = country.findall('./population[last()]')
    groups = country.findall('ethnicgroup')
    for pop in pops:
        for group in groups:
            populations.append(int(pop.text))
            ethnic_groups.append(group.text), 
            percentages.append(float(group.get('percentage')))

In [12]:
#dataframe for manipulation
df = pd.DataFrame({'population': populations, 'group': ethnic_groups, 'percentage': percentages})

In [13]:
#create new column to give size of ethnic group population in each country
df['ethnic population'] = ((df['percentage'] / 100) * df['population'])

In [14]:
#remove now unecessary columns
del df['percentage']
del df['population']

In [15]:
#group the data by ethnic groups
df = df.astype({'ethnic population': int})
df = df.groupby(df['group']).sum()

In [16]:
#get the top 10
df.sort_values('ethnic population', ascending = False).head(10)

,ethnic population
group,
Han Chinese,1245058800
Indo-Aryan,871815583
European,494872201
African,318325104
Dravidian,302713744
Mestizo,157734349
Bengali,146776916
Russian,131856989
Japanese,126534212


## 4) Find the name and country of:
### a) the longest river

In [17]:
#for 'river', 'lake' and 'airport', I first checked available tags for query with .iterchildren() on each

In [18]:
#work backwards from length through use of iterancestors to ensure only rivers with length value are included
lengths = document.findall('./river/length')

In [19]:
country_names = []
river_names = []
river_lengths = []

for length in lengths:
    for river in length.iterancestors('river'):
        co_code = river.get('country') #this attribute is a country code or multiple codes
        multi_co_code = co_code.split() #where there are multiple codes,each country needs individual river entity
        
        for code in multi_co_code:
            for country in document.findall('country'):
                car_code = country.get('car_code')
                
                if car_code == code:
                    country_names.append(country.find('name').text)
                    river_names.append(river.find('name').text)
                    river_lengths.append(float((river.find('length').text)))

In [20]:
#read all into Pandas dataframe
df = pd.DataFrame({'country': country_names, 'river name': river_names, 'river length': river_lengths})
df.head()

,country,river length,river name
0,Iceland,230.0,Thjorsa
1,Iceland,206.0,Joekulsa a Fjoellum
2,Norway,604.0,Glomma
3,Norway,322.0,Lagen
4,Sweden,93.0,Goetaaelv


In [21]:
#find the longest river and name of it
longest = df.sort_values('river length', ascending = False).head(1)
river_name_longest = longest.iloc[0, 2]

In [22]:
#answer is new dataframe just including countries associated with the name of longest river
ans_df = df[df['river name'] == river_name_longest]
ans_df

,country,river length,river name
298,Colombia,6448.0,Amazonas
299,Brazil,6448.0,Amazonas
300,Peru,6448.0,Amazonas


### b) the largest lake

In [23]:
#alteration of variables from River code above

In [24]:
areas = document.findall('./lake/area')

In [25]:
country_names = []
lake_names = []
lake_areas = []

for area in areas:
    for lake in area.iterancestors('lake'):
        co_code = lake.get('country') #this attribute is a country code or multiple codes
        multi_co_code = co_code.split() #where there are multiple codes,each country needs individual lake entity
        
        for code in multi_co_code:
            for country in document.findall('country'):
                car_code = country.get('car_code')
                
                if car_code == code:
                    country_names.append(country.find('name').text)
                    lake_names.append(lake.find('name').text)
                    lake_areas.append(float((lake.find('area').text)))

In [26]:
df = pd.DataFrame({'country': country_names, 'lake name': lake_names, 'lake area': lake_areas})
df.head()

,country,lake area,lake name
0,Finland,1040.0,Inari
1,Finland,928.0,Oulujaervi
2,Finland,472.0,Kallavesi
3,Finland,4370.0,Saimaa
4,Finland,1118.0,Paeijaenne


In [27]:
largest = df.sort_values('lake area', ascending = False).head(1)
lake_name_largest = largest.iloc[0, 2]

In [28]:
ans_df = df[df['lake name'] == lake_name_largest]
ans_df

,country,lake area,lake name
68,Russia,386400.0,Caspian Sea
69,Azerbaijan,386400.0,Caspian Sea
70,Kazakhstan,386400.0,Caspian Sea
71,Iran,386400.0,Caspian Sea
72,Turkmenistan,386400.0,Caspian Sea


### c) the airport at highest elevation

In [29]:
#abbreviated version of River and Lake code above; no requirement for listing multiple codes/countries

In [30]:
elevations = document.findall('./airport/elevation')

In [31]:
country_names = []
airport_names = []
airport_elevations = []

for elevation in elevations:
    for airport in elevation.iterancestors('airport'):
        co_code = airport.get('country') #this attribute is a country code or multiple codes
        multi_co_code = co_code.split() #where there are multiple codes,each country needs individual airport entity
        
        for code in multi_co_code:
            for country in document.findall('country'):
                car_code = country.get('car_code')
                
                if car_code == code:
                    country_names.append(country.find('name').text)
                    airport_names.append(airport.find('name').text)
                    airport_elevations.append((airport.find('elevation').text))

In [32]:
df = pd.DataFrame({'country': country_names, 'airport name': airport_names, 'airport elevation': airport_elevations})
df = df.astype({'airport elevation': float}).dropna()
df.head()

,airport elevation,airport name,country
0,977.0,Herat,Afghanistan
1,1792.0,Kabul Intl,Afghanistan
2,38.0,Tirana Rinas,Albania
3,811.0,Cheikh Larbi Tebessi,Algeria
4,822.0,Batna Airport,Algeria


In [33]:
highest = df.sort_values('airport elevation', ascending = False).head(1)
airport_name_highest = highest.iloc[0, 1]

In [34]:
ans_df = df[df['airport name'] == airport_name_highest]
ans_df

,airport elevation,airport name,country
80,4063.0,El Alto Intl,Bolivia
